In [1]:
from bsf_env import init_spark, init_mariadb_engine,set_spark_verbosity
from pyspark.sql.functions import lit, current_timestamp
import pandas as pd
import numpy as np
from pyspark.sql.types import *
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from IPython.display import display, HTML

spark = init_spark("bsf_candidates_analysis", log_level="WARN", show_progress=False, enable_ui=True)
engine = init_mariadb_engine()

ingest_ts = spark.sql("SELECT current_timestamp()").collect()[0][0]




:: loading settings :: url = jar:file:/home/jupyter/.venv/python3.9_bsf/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jupyter/.ivy2/cache
The jars for the packages stored in: /home/jupyter/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c205ab18-2d06-4aca-9e61-56c61adfa08b;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0rc1 in spark-list
	found io.delta#delta-storage;3.0.0rc1 in spark-list
	found org.antlr#antlr4-runtime;4.9.3 in spark-list
:: resolution report :: resolve 396ms :: artifacts dl 15ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.0.0rc1 from spark-list in [default]
	io.delta#delta-storage;3.0.0rc1 from spark-list in [default]
	org.antlr#antlr4-runtime;4.9.3 from spark-list in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	

[Spark] Started 'bsf_candidates_analysis' log_level=WARN (effective=WARN), progress=False


In [2]:
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", 200)         # Adjust width for readability
pd.set_option("display.max_rows", 20)       # Show only top 20 rows by default

In [5]:
from pyspark.sql import functions as F

# Show tables
tables_df = spark.sql("SHOW TABLES IN bsf")
tables_df.show(truncate=False)

# Add row count for each table
for row in tables_df.collect():
    table_name = row['tableName']
    full_name = f"bsf.{table_name}"
    
    try:
        count = spark.table(full_name).count()
    except Exception as e:
        count = f"Error: {e}"
    
    print(f"Table: {full_name} | Rows: {count}")



25/09/08 21:44:32 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/09/08 21:44:32 WARN HiveConf: HiveConf of name hive.metastore.client.connect.timeout does not exist
25/09/08 21:44:32 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/09/08 21:44:32 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/09/08 21:44:32 WARN HiveConf: HiveConf of name hive.metastore.client.connect.timeout does not exist
25/09/08 21:44:32 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/09/08 21:44:34 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/09/08 21:44:34 WARN HiveConf: HiveConf of name hive.metastore.client.connect.timeout does not exist
25/09/08 21:44:34 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/09/08 21:44:37 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+---------+-------------------------+-----------+
|namespace|tableName                |isTemporary|
+---------+-------------------------+-----------+
|bsf      |company                  |false      |
|bsf      |companystockhistory      |false      |
|bsf      |daily_signals            |false      |
|bsf      |daily_signals_allcol     |false      |
|bsf      |daily_signals_last       |false      |
|bsf      |daily_signals_last_allcol|false      |
+---------+-------------------------+-----------+

Table: bsf.company | Rows: 30949
Table: bsf.companystockhistory | Rows: 461424
Table: bsf.daily_signals | Rows: 1844868
Table: bsf.daily_signals_allcol | Rows: 1844868
Table: bsf.daily_signals_last | Rows: 7372
Table: bsf.daily_signals_last_allcol | Rows: 7372


In [ ]:
from pyspark.sql import functions as F

df = spark.table("bsf.daily_signals")

In [44]:
df1 = spark.table("bsf.daily_signals_last_allcol ")

In [45]:
pdf1 = df1.toPandas()
display(pdf1)

,CompanyId,StockDate,Open,High,Low,Close,Doji,Hammer,HangingMan,InvertedHammer,ShootingStar,BullishMarubozu,BearishMarubozu,SuspiciousCandle,BullishEngulfing,BearishEngulfing,BullishHarami,BearishHarami,HaramiCross,PiercingLine,DarkCloudCover,MorningStar,EveningStar,ThreeWhiteSoldiers,ThreeBlackCrows,TweezerTop,TweezerBottom,InsideBar,OutsideBar,NearHigh,NearLow,PatternCount,PatternType,MA,MA_slope,UpTrend_MA,DownTrend_MA,RecentReturn,UpTrend_Return,DownTrend_Return,Volatility,LowVolatility,HighVolatility,ROC,MomentumUp,MomentumDown,ConfirmedUpTrend,ConfirmedDownTrend,ValidHammer,ValidBullishEngulfing,ValidPiercingLine,ValidMorningStar,ValidThreeWhiteSoldiers,ValidBullishMarubozu,ValidTweezerBottom,ValidShootingStar,ValidBearishEngulfing,ValidDarkCloud,ValidEveningStar,ValidThreeBlackCrows,ValidBearishMarubozu,ValidTweezerTop,ValidHaramiCross,ValidBullishHarami,ValidBearishHarami,ValidInsideBar,ValidOutsideBar,ValidCustomSignal1,ValidCustomSignal2,TomorrowClose,TomorrowReturn,Return,AvgReturn,MomentumZ,BuyThresh,SellThresh,MomentumAction,BullScore,BearScore,PatternScore,PatternScoreNorm,PatternAction,CandleAction,CandidateAction,Action,TomorrowAction,TomorrowActionSource,SignalStrengthHybrid,ActionConfidence,BullishStrengthHybrid,BearishStrengthHybrid,SignalDuration,ValidAction,HasValidSignal,SignalStrength,BatchId,IngestedAt,TimeFrame
0,73855,2025-09-05,0.09100,0.0940,0.08600,0.09040,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True,3,ThreeWhiteSoldiers,0.086717,0.017680,True,False,0.118812,True,False,0.087432,False,True,0.118812,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,0.000221,0.014486,-0.163151,0.046194,-0.028567,Sell,8.0,17.0,-9.0,-0.9,Sell,Hold,Sell,Sell,Hold,LastRowHold,0.289331,0.289331,0.289331,0.289331,172.0,True,True,1,73855_20250908_231622,20250908_231622,Long
1,30947,2025-09-05,0.00230,0.0049,0.00230,0.00370,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True,3,ThreeWhiteSoldiers,0.003660,0.019499,True,False,0.193548,True,False,0.252307,False,True,0.193548,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,0.057143,0.045986,0.044221,0.055701,-0.020080,Hold,6.0,19.0,-13.0,-1.3,Sell,Hold,Hold,Hold,Hold,LastRowHold,0.401552,0.000000,0.401552,0.401552,163.0,False,True,0,30947_20250908_230927,20250908_230927,Long
2,59234,2025-09-05,0.00646,0.0071,0.00575,0.00700,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,2,ThreeWhiteSoldiers,0.005811,0.507393,True,False,0.707317,True,False,0.107620,True,False,0.707317,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,0.060606,0.059593,0.009415,0.030271,-0.045324,Hold,30.0,0.0,30.0,3.0,Buy,Hold,Hold,Hold,Hold,LastRowHold,0.919959,0.000000,0.919959,0.919959,200.0,False,True,0,59234_20250908_231400,20250908_231400,Long
3,103448,2025-09-05,0.01500,0.0200,0.00900,0.01965,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True,3,ThreeWhiteSoldiers,0.008993,0.803725,True,False,0.310000,True,False,15.987399,False,True,0.965000,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,0.310000,6.872905,-0.410505,0.009245,-0.048414,Sell,3.0,0.0,3.0,3.0,Buy,Hold,Buy,Hold,Hold,LastRowHold,0.583386,0.000000,0.583386,0.583386,194.0,False,True,0,103448_20250908_235630,20250908_235630,Daily
4,88988,2025-0

In [47]:
df1.groupBy("Action").count().orderBy(F.desc("count")).show(truncate=False)

+------+-----+
|Action|count|
+------+-----+
|Hold  |5248 |
|Buy   |1182 |
|Sell  |942  |
+------+-----+



In [48]:
df1.groupBy("TimeFrame", "Action") \
  .count() \
  .orderBy("TimeFrame", F.desc("count")) \
  .show(truncate=False)
print(df1)

+---------+------+-----+
|TimeFrame|Action|count|
+---------+------+-----+
|Daily    |Hold  |1321 |
|Daily    |Buy   |289  |
|Daily    |Sell  |233  |
|Long     |Hold  |1331 |
|Long     |Buy   |293  |
|Long     |Sell  |219  |
|Short    |Hold  |1317 |
|Short    |Buy   |272  |
|Short    |Sell  |254  |
|Swing    |Hold  |1279 |
|Swing    |Buy   |328  |
|Swing    |Sell  |236  |
+---------+------+-----+

DataFrame[CompanyId: bigint, StockDate: date, Open: double, High: double, Low: double, Close: double, Doji: boolean, Hammer: boolean, HangingMan: boolean, InvertedHammer: boolean, ShootingStar: boolean, BullishMarubozu: boolean, BearishMarubozu: boolean, SuspiciousCandle: boolean, BullishEngulfing: boolean, BearishEngulfing: boolean, BullishHarami: boolean, BearishHarami: boolean, HaramiCross: boolean, PiercingLine: boolean, DarkCloudCover: boolean, MorningStar: boolean, EveningStar: boolean, ThreeWhiteSoldiers: boolean, ThreeBlackCrows: boolean, TweezerTop: boolean, TweezerBottom: boolean, I

In [37]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# -----------------------------
# Aggregate Buy/Sell/Hold counts per company per timeframe
# -----------------------------
df_counts = df.groupBy("CompanyId", "TimeFrame").agg(
    F.sum(F.when(F.col("Action") == "Buy", 1).otherwise(0)).alias("BuyCount"),
    F.sum(F.when(F.col("Action") == "Sell", 1).otherwise(0)).alias("SellCount"),
    F.sum(F.when(F.col("Action") == "Hold", 1).otherwise(0)).alias("HoldCount")
)

# -----------------------------
# Rank companies within each timeframe by BuyCount
# -----------------------------
window_spec = Window.partitionBy("TimeFrame").orderBy(F.desc("BuyCount"))
df_ranked = df_counts.withColumn("rank", F.row_number().over(window_spec))

# -----------------------------
# Keep only top 3 companies per timeframe
# -----------------------------
top3_companies = df_ranked.filter(F.col("rank") <= 3).select("CompanyId", "TimeFrame")

# -----------------------------
# Join back to the original df to get full rows for those top 3 companies
# -----------------------------
df_top3_rows = df.join(top3_companies, on=["CompanyId", "TimeFrame"], how="inner")

In [34]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# -----------------------------
# Define a window partitioned by TimeFrame and CompanyId, ordered by StockDate descending
# -----------------------------
window_spec = Window.partitionBy("TimeFrame", "CompanyId").orderBy(F.desc("StockDate"))

# -----------------------------
# Get the latest row per company per timeframe
# -----------------------------
df_latest = df.withColumn("row_num", F.row_number().over(window_spec)) \
              .filter(F.col("row_num") == 1) \
              .drop("row_num")

# -----------------------------
# Filter for companies where TomorrowAction is 'Buy'
# -----------------------------
df_buy_latest = df_latest.filter(F.col("TomorrowAction") == "Buy")
display(df_buy_latest.toPandas())

,CompanyId,StockDate,Open,High,Low,Close,TomorrowClose,Return,TomorrowReturn,Doji,Hammer,InvertedHammer,ShootingStar,BullishEngulfing,BearishEngulfing,PiercingLine,DarkCloudCover,MorningStar,EveningStar,ThreeWhiteSoldiers,ThreeBlackCrows,TweezerTop,TweezerBottom,InsideBar,OutsideBar,MA,MA_slope,UpTrend_MA,DownTrend_MA,MomentumUp,MomentumDown,ConfirmedUpTrend,ConfirmedDownTrend,RecentReturn,UpTrend_Return,DownTrend_Return,Volatility,LowVolatility,HighVolatility,ROC,MomentumZ,SignalStrength,SignalStrengthHybrid,ActionConfidence,BullishStrengthHybrid,BearishStrengthHybrid,SignalDuration,ValidAction,HasValidSignal,MomentumAction,PatternAction,CandleAction,CandidateAction,Action,TomorrowAction,TomorrowActionSource,BatchId,IngestedAt,TimeFrame


In [35]:
# -----------------------------
# Rank by Return descending per timeframe
# -----------------------------
window_rank = Window.partitionBy("TimeFrame").orderBy(F.desc("Return"))
df_ranked = df_buy_latest.withColumn("rank", F.row_number().over(window_rank))

# -----------------------------
# Keep top 3 companies per timeframe
# -----------------------------
top3_companies = df_ranked.filter(F.col("rank") <= 10).select("CompanyId", "TimeFrame", "Return", "TomorrowAction")

# -----------------------------
# Join back to get full historical rows for these top companies
# -----------------------------
df_top3_rows = df.join(top3_companies, on=["CompanyId", "TimeFrame"], how="inner")

In [31]:
# -----------------------------
# Optional: convert each timeframe to Pandas
# -----------------------------
pdf_short = df_top3_rows.filter(F.col("TimeFrame") == "Short").toPandas()
pdf_swing = df_top3_rows.filter(F.col("TimeFrame") == "Swing").toPandas()
pdf_long = df_top3_rows.filter(F.col("TimeFrame") == "Long").toPandas()
pdf_daily = df_top3_rows.filter(F.col("TimeFrame") == "Daily").toPandas()
# -----------------------------
# Save to CSV without the index
# -----------------------------
pdf_short.to_csv(f"cvs/short_top3_output.csv", index=False)
pdf_swing.to_csv(f"cvs/swing_top3_output.csv", index=False)
pdf_long.to_csv(f"cvs/long_top3_output.csv", index=False)
pdf_daily.to_csv(f"cvs/daily_top3_output.csv", index=False)

In [36]:
display(top3_companies.toPandas())

,CompanyId,TimeFrame,Return,TomorrowAction


In [26]:
#List of timeframes
timeframes = ["Short", "Swing", "Long", "Daily"]

timeframe_dfs = {
    "Short": pdf_short,
    "Swing": pdf_swing,
    "Long": pdf_long,
    "Daily": pdf_daily
}


In [21]:
display(pdf_short)

,CompanyId,TimeFrame,StockDate,Open,High,Low,Close,TomorrowClose,Return,TomorrowReturn,Doji,Hammer,InvertedHammer,ShootingStar,BullishEngulfing,BearishEngulfing,PiercingLine,DarkCloudCover,MorningStar,EveningStar,ThreeWhiteSoldiers,ThreeBlackCrows,TweezerTop,TweezerBottom,InsideBar,OutsideBar,MA,MA_slope,UpTrend_MA,DownTrend_MA,MomentumUp,MomentumDown,ConfirmedUpTrend,ConfirmedDownTrend,RecentReturn,UpTrend_Return,DownTrend_Return,Volatility,LowVolatility,HighVolatility,ROC,MomentumZ,SignalStrength,SignalStrengthHybrid,ActionConfidence,BullishStrengthHybrid,BearishStrengthHybrid,SignalDuration,ValidAction,HasValidSignal,MomentumAction,PatternAction,CandleAction,CandidateAction,Action,TomorrowAction,TomorrowActionSource,BatchId,IngestedAt
0,58954,Short,2024-07-26,0.2247,0.2280,0.20840,0.21415,0.21304,0.000000,-0.005183,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN,NaN,False,False,False,False,False,False,NaN,False,False,NaN,False,False,NaN,NaN,0,NaN,0.000000,NaN,NaN,1.0,False,False,Hold,Hold,Hold,Hold,Hold,Hold,Hold(no_signal),58954_20250908_220144,20250908_220144
1,58954,Short,2024-07-29,0.2243,0.2320,0.21304,0.21304,0.21610,-0.005183,0.014363,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.213595,NaN,False,False,False,False,False,False,-0.005183,False,True,0.003665,False,False,NaN,-0.707107,0,0.069783,0.000000,0.069783,0.069783,1.0,False,True,Sell,Hold,Hold,Hold,Hold,Hold,Hold(no_signal),58954_20250908_220144,20250908_220144
2,58954,Short,2024-07-30,0.2101,0.2264,0.21000,0.21610,0.21260,0.014363,-0.016196,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.214570,NaN,False,False,False,False,False,False,0.014363,True,False,0.010126,False,False,0.009106,1.116243,0,0.110161,0.000000,0.110161,0.110161,1.0,False,True,Buy,Hold,Hold,Hold,Hold,Buy,NextAction(filtered),58954_20250908_220144,20250908_220144
3,58954,Short,2024-07-31,0.2228,0.2281,0.21260,0.21260,0.20570,-0.016196,-0.032455,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.214350,0.003535,True,False,False,False,False,False,-0.016196,False,True,0.012691,True,False,-0.002065,-1.137984,1,0.210995,0.210995,0.210995,0.210995,2.0,True,True,Sell,Buy,Hold,Buy,Buy,Sell,NextAction(filtered),58954_20250908_220144,20250908_220144
4,58954,Short,2024-08-01,0.2040,0.2057,0.20000,0.20570,0.20500,-0.032455,-0.003403,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,0.209150,-0.025260,False,True,False,True,False,True,-0.032455,False,True,0.017587,True,False,-0.048126,-1.396527,1,0.335199,0.335199,0.335199,0.335199,3.0,True,True,Sell,Sell,Hold,Sell,Sell,Buy,NextAction(filtered),58954_20250908_220144,20250908_220144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14020,84236,Short,2025-08-29,0.0032,0.0040,0.00320,0.00350,0.00380,-0.125000,0.085714,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,0.003750,-0.062500,False,True,False,True,False,True,-0.125000,False,True,0.085399,True,False,-0.125000,-1.501890,1,0.390533,0.390533,0.390533,0.390533,182.0,True,True,Sell,Sell,Hold,Sell,Sell,Buy,NextAction(filtered),84236_20250908_220558,20250908_220558
14021,84236,Short,2025-09-02,0.0036,0.0040,0.00320,0.00380,0.00370,0.085714,-0.026316,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,0.003650,-0.087500,False,True,False,True,False,False,0.085714,True,False,0.077902,True,False,-0.050000,0.801695,3,1.169955,1.169955,1.169955,0.503289,183.0,True,True,Buy,Sell,Buy,Buy,Buy,Sell,NextAction(filtered),84236_20250908_220558,20250908_220558
14022,84236,Short,2025-09-03,0.0033,0.0037,0.00220,

In [29]:
import pandas as pd

all_top_gainers = []  # List to collect DataFrames

for tf, pdf in timeframe_dfs.items():
    print(f"\n=== Processing timeframe: {tf} ===")
    
    pdf_tf = pdf.copy()
    
    # Log-transform features
    for col in ["Open", "High", "Low", "Close"]:
        pdf_tf[f"log_{col}"] = np.log(pdf_tf[col].replace(0, epsilon))
    pdf_tf["log_TomorrowClose"] = np.log(pdf_tf["TomorrowClose"].replace(0, epsilon))
    
    # Historical rows
    hist_mask = pdf_tf["TomorrowClose"].notna()
    model_df = pdf_tf[hist_mask].copy()
    if model_df.empty:
        print("⚠️ No historical data, skipping timeframe.")
        continue

    X = model_df[features].fillna(0)
    y = model_df["log_TomorrowClose"]

    # Train model
    ols = LinearRegression()
    ols.fit(X, y)

    # Predict for latest rows
    future_mask = pdf_tf["TomorrowClose"].isna()
    latest_rows = (
        pdf_tf[future_mask]
        .sort_values("StockDate")
        .groupby("CompanyId", as_index=False)
        .tail(1)
    )
    if not latest_rows.empty:
        latest_X = latest_rows[features].fillna(0)
        latest_rows["PredictedTomorrowClose"] = np.exp(ols.predict(latest_X))
        latest_rows["PredictedReturn"] = (
            latest_rows["PredictedTomorrowClose"] - latest_rows["Close"]
        ) / latest_rows["Close"]
        latest_rows["TimeFrame"] = tf  # Add timeframe column
        
        # Keep only valid predictions
        valid_preds = latest_rows[latest_rows["PredictedTomorrowClose"].notna()]
        all_top_gainers.append(valid_preds)

# Concatenate all timeframes
combined_df = pd.concat(all_top_gainers, ignore_index=True)

# Display top 20 by PredictedReturn across all timeframes
combined_df.sort_values("PredictedReturn", ascending=False, inplace=True)

# Sort by PredictedReturn descending (highest return first)
top_candidates = combined_df.sort_values("PredictedReturn", ascending=False)

# Display top 20
display(top_candidates.head(10))
top_candidates.to_csv(f"cvs/top_candidates_output.csv", index=False)



=== Processing timeframe: Short ===

=== Processing timeframe: Swing ===

=== Processing timeframe: Long ===

=== Processing timeframe: Daily ===


,CompanyId,TimeFrame,StockDate,Open,High,Low,Close,TomorrowClose,Return,TomorrowReturn,Doji,Hammer,InvertedHammer,ShootingStar,BullishEngulfing,BearishEngulfing,PiercingLine,DarkCloudCover,MorningStar,EveningStar,ThreeWhiteSoldiers,ThreeBlackCrows,TweezerTop,TweezerBottom,InsideBar,OutsideBar,MA,MA_slope,UpTrend_MA,DownTrend_MA,MomentumUp,MomentumDown,ConfirmedUpTrend,ConfirmedDownTrend,RecentReturn,UpTrend_Return,DownTrend_Return,Volatility,LowVolatility,HighVolatility,ROC,MomentumZ,SignalStrength,SignalStrengthHybrid,ActionConfidence,BullishStrengthHybrid,BearishStrengthHybrid,SignalDuration,ValidAction,HasValidSignal,MomentumAction,PatternAction,CandleAction,CandidateAction,Action,TomorrowAction,TomorrowActionSource,BatchId,IngestedAt,log_Open,log_High,log_Low,log_Close,log_TomorrowClose,PredictedTomorrowClose,PredictedReturn
9,35715,Short,2025-09-05,0.00177,0.00240,0.001500,0.001500,NaN,-0.347826,NaN,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.001900,0.055556,True,False,True,False,False,False,-0.347826,False,True,0.432309,False,True,0.071429,-0.907883,0,0.277164,0.000000,0.277164,0.277164,192.0,False,True,Sell,Buy,Hold,Buy,Hold,Hold,LastRowHold,35715_20250908_215835,20250908_215835,-6.336776,-6.032287,-6.502290,-6.502290,NaN,0.001854,0.235808
12,102601,Short,2025-09-05,0.01650,0.01960,0.012101,0.012101,NaN,-0.182365,NaN,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,0.013450,-0.103300,False,True,False,True,False,True,-0.182365,False,True,0.149555,True,False,-0.193267,-1.368837,2,0.498089,0.498089,0.498089,0.498089,191.0,True,True,Sell,Sell,Hold,Sell,Sell,Hold,LastRowHold,102601_20250908_220944,20250908_220944,-4.104395,-3.932226,-4.414467,-4.414467,NaN,0.014596,0.206184
2,644,Short,2025-09-05,0.06300,0.07400,0.050000,0.050000,NaN,-0.200000,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.056250,0.086957,True,False,True,False,False,False,-0.200000,False,True,0.237865,False,True,0.219512,-0.830668,1,0.287963,0.287963,0.287963,0.287963,194.0,True,True,Sell,Sell,Hold,Sell,Sell,Hold,LastRowHold,644_20250908_215519,20250908_215519,-2.764621,-2.603690,-2.995732,-2.995732,NaN,0.057511,0.150218
108,38954,Long,2025-09-05,0.05000,0.07800,0.040000,0.043250,NaN,-0.049451,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.035990,0.402353,True,False,True,False,True,False,0.740443,True,False,0.172400,False,True,0.740443,-0.689237,1,0.177084,0.177084,0.177084,0.177084,200.0,True,True,Sell,Buy,Hold,Buy,Buy,Hold,LastRowHold,38954_20250908_231106,20250908_231106,-2.995732,-2.551046,-3.218876,-3.140758,NaN,0.049289,0.139631
59,38954,Swing,2025-09-05,0.05000,0.07800,0.040000,0.043250,NaN,-0.049451,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.044880,0.656089,True,False,True,False,True,False,0.253623,True,False,0.172400,False,True,0.253623,-0.689237,0,0.935604,0.000000,0.935604,0.935604,208.0,False,True,Sell,Buy,Hold,Buy,Hold,Hold,LastRowHold,38954_20250908_223554,20250908_223554,-2.995732,-2.551046,-3.218876,-3.140758,NaN,0.049231,0.138284
11,44177,Short,2025-09-05,0.04600,0.07248,0.035000,0.049000,NaN,0.323609,NaN,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,0.043010,0.211549,True,False,True,False,True,False,0.323609,True,False,0.160009,False,True,0.392045,1.643509,0,0.831425,0.000000,0.831425,0.831425,199.0,False,True,Buy,Buy,Hold,Buy,Hold,Hold,LastRowHold,44177_20250908_215952,20250908_215952,-3.079114,-2.624445,-3.352407,-3.015935,NaN,0.054727,0.116869
174,644,Daily,2025-09-05,0.06300,0.07400,0.050000,0.050000,NaN,-0.200000,NaN,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,0.059491,-0.076589,False,True,False,True,False,True,-0.200000,False,True,0.237865,False,True,-0.295775,-0.830668,1,0.

In [17]:
filtered_gainers = top_gainers[
    (top_gainers["PredictedReturn"] > 0.05) &
    (top_gainers["TomorrowAction"] == "Buy") &
    (top_gainers["ActionConfidence"] > 0.5)
]
display(filtered_gainers)

NameError: name 'top_gainers' is not defined

In [ ]:
spark.stop()